<a href="https://colab.research.google.com/github/wothmag07/genai-bootcamp/blob/main/BERTFinetuning_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q BitsandBytes transformers datasets seqeval evaluate accelerate tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00


In [2]:
import datasets
from transformers import BertTokenizerFast, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate
import numpy as np
from transformers import pipeline

seqeval = evaluate.load("seqeval")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
dataset = datasets.load_dataset("eriktks/conll2003", trust_remote_code=True)

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

**CoNLL-2003 NER Dataset**

The CoNLL-2003 dataset is a widely used benchmark for NER, featuring language-independent named entity recognition. It includes four entity types:

* PER (Persons)
* LOC (Locations)
* ORG (Organizations)
* MISC (Miscellaneous entities)

**Dataset Format**

Each data sample follows a structured format with four columns:

* Word - The actual token in the sentence.
* POS Tag - The part-of-speech tag.
* Chunk Tag - The syntactic chunking label.
* NER Tag - The named entity label in IOB2 format.

**IOB2 Tagging Scheme**

The dataset follows the IOB2 tagging scheme:

* B-TYPE (Beginning) - Marks the first word of a named entity.
* I-TYPE (Inside) - Marks subsequent words of a named entity.
* O (Outside) - Indicates words that are not part of any named entity.

Each word is placed on a separate line, and sentences are separated by empty lines.

This dataset is commonly used for training NER models with deep learning and machine learning techniques, including LSTMs, CRFs, and Transformers (like BERT).

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
dataset['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [6]:
eg = dataset['train'][70]
eg

{'id': '70',
 'tokens': ['GOV',
  'LAW',
  'GERMAN',
  'HOME',
  'CTRY',
  '=',
  'TAX',
  'PROVS',
  'STANDARD'],
 'pos_tags': [22, 22, 22, 22, 22, 34, 21, 24, 38],
 'chunk_tags': [11, 12, 12, 12, 12, 21, 11, 12, 21],
 'ner_tags': [0, 0, 7, 0, 0, 0, 0, 0, 0]}

In [7]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

This dictionary describes the schema of a dataset, likely from a Natural Language Processing (NLP) task involving tokenization, Part-of-Speech (POS) tagging, chunking, and Named Entity Recognition (NER). Let’s break it down:

1. id
Type: string
Represents a unique identifier for each data sample.
2. tokens
Type: Sequence(Value(dtype='string'))
A sequence of word tokens from the text.
3. pos_tags (Part-of-Speech Tags)
Type: Sequence(ClassLabel(...))
Each token has a corresponding POS tag based on the Penn Treebank POS tagset, which includes:
NN (noun, singular)
VB (verb, base form)
JJ (adjective)
IN (preposition), etc.
POS tagging helps in syntactic and semantic analysis of sentences.
4. chunk_tags (Syntactic Chunking)
Type: Sequence(ClassLabel(...))
Indicates phrase chunks (e.g., noun phrases, verb phrases) using Inside-Outside-Beginning (IOB) tagging:
B-NP (Begin noun phrase)
I-NP (Inside noun phrase)
B-VP (Begin verb phrase)
I-VP (Inside verb phrase)
O (Outside any chunk)
Helps in grouping words into meaningful phrases.
5. ner_tags (Named Entity Recognition)
Type: Sequence(ClassLabel(...))
Labels each token with an NER tag, identifying named entities like:
B-PER (Beginning of a person’s name)
I-PER (Inside a person’s name)
B-ORG (Beginning of an organization)
I-ORG (Inside an organization)
B-LOC (Beginning of a location)
I-LOC (Inside a location)
O (Outside any entity)
This helps in recognizing important entities in text.

In [8]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", padding=True, truncation=True)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [24]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [9]:
tokenized_ids = tokenizer(eg["tokens"],is_split_into_words=True)
print(tokenized_ids)
tokens = tokenizer.convert_ids_to_tokens(tokenized_ids["input_ids"])
print(tokens)

{'input_ids': [101, 18079, 2375, 2446, 2188, 14931, 2854, 1027, 4171, 4013, 15088, 3115, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'gov', 'law', 'german', 'home', 'ct', '##ry', '=', 'tax', 'pro', '##vs', 'standard', '[SEP]']


In [10]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)
ner_pipeline("My name is Wolfgang and I live in Berlin")

Device set to use cuda:0


[{'entity': 'LABEL_0',
  'score': np.float32(0.17899619),
  'index': 1,
  'word': 'my',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_0',
  'score': np.float32(0.1570979),
  'index': 2,
  'word': 'name',
  'start': 3,
  'end': 7},
 {'entity': 'LABEL_4',
  'score': np.float32(0.14485478),
  'index': 3,
  'word': 'is',
  'start': 8,
  'end': 10},
 {'entity': 'LABEL_0',
  'score': np.float32(0.17015001),
  'index': 4,
  'word': 'wolfgang',
  'start': 11,
  'end': 19},
 {'entity': 'LABEL_0',
  'score': np.float32(0.17837398),
  'index': 5,
  'word': 'and',
  'start': 20,
  'end': 23},
 {'entity': 'LABEL_0',
  'score': np.float32(0.1942502),
  'index': 6,
  'word': 'i',
  'start': 24,
  'end': 25},
 {'entity': 'LABEL_4',
  'score': np.float32(0.13694938),
  'index': 7,
  'word': 'live',
  'start': 26,
  'end': 30},
 {'entity': 'LABEL_0',
  'score': np.float32(0.15266135),
  'index': 8,
  'word': 'in',
  'start': 31,
  'end': 33},
 {'entity': 'LABEL_0',
  'score': np.float32(0.21886183),
  '

In [11]:
def label_tokenize(samples, label_all_tokens=True):
  """
  Tokenizes and aligns labels with sub-tokens for a given dataset sample.
  """

  # Tokenization (no change)
  tokenized_inputs = tokenizer(samples['tokens'], truncation=True, is_split_into_words=True)
  labels = []

  for i, label in enumerate(samples['ner_tags']):
      word_ids = tokenized_inputs.word_ids(batch_index=i)

      prev_word_index = None
      label_ids = []

      for word_index in word_ids:
          if word_index is None:
              label_ids.append(-100)
          elif word_index != prev_word_index:
              label_ids.append(label[word_index])
          else:
              label_ids.append(label[word_index] if label_all_tokens else -100)
          prev_word_index = word_index
      labels.append(label_ids)
  # Return (no change)
  tokenized_inputs['labels'] = labels # changed to label_ids
  return tokenized_inputs

In [12]:
q = label_tokenize(samples=dataset['train'][0:1])
q

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]]}

In [13]:
q["labels"][0]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

In [14]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
eu______________________________________ 3
rejects_________________________________ 0
german__________________________________ 7
call____________________________________ 0
to______________________________________ 0
boycott_________________________________ 0
british_________________________________ 7
lamb____________________________________ 0
._______________________________________ 0
[SEP]___________________________________ -100


In [15]:
tokenized_ds = dataset.map(label_tokenize, batched=True)
tokenized_ds

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

Listing the labels

In [16]:
label_list=dataset["train"].features["ner_tags"].feature.names
label_id_dict = {i: label for i, label in enumerate(label_list)}
label_id_dict


{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

Lets consider our eg sample

In [17]:
eg

{'id': '70',
 'tokens': ['GOV',
  'LAW',
  'GERMAN',
  'HOME',
  'CTRY',
  '=',
  'TAX',
  'PROVS',
  'STANDARD'],
 'pos_tags': [22, 22, 22, 22, 22, 34, 21, 24, 38],
 'chunk_tags': [11, 12, 12, 12, 12, 21, 11, 12, 21],
 'ner_tags': [0, 0, 7, 0, 0, 0, 0, 0, 0]}

In [18]:
labels = [label_list[i] for i in eg["ner_tags"]]
labels

['O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O']

In [19]:
# for param in model.parameters():
#   print(param)

In [20]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_id_dict[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_id_dict[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [21]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

args = TrainingArguments("valid-ner",
                         evaluation_strategy = "epoch",
                         learning_rate=2e-5,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                         num_train_epochs=20,
                         weight_decay=0.01,
                         report_to="none")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-21-72ef6009d264>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.218300,0.062880,0.924106,0.927621,0.925860,0.982700
2,0.045600,0.059024,0.934952,0.939031,0.936987,0.984924
3,0.026200,0.060108,0.931853,0.943842,0.937809,0.985448
4,0.015800,0.062208,0.935491,0.950666,0.943017,0.986369
5,0.010000,0.066559,0.939754,0.947533,0.943627,0.986290
6,0.006700,0.071191,0.934208,0.948316,0.941209,0.985861
7,0.005000,0.078597,0.942524,0.948428,0.945467,0.986401
8,0.004400,0.077937,0.941189,0.952456,0.946789,0.987068
9,0.003100,0.082195,0.944754,0.946974,0.945863,0.986576
10,0.003100,0.080321,0.944161,0.951449,0.947791,0.987100


TrainOutput(global_step=17560, training_loss=0.014386605999298852, metrics={'train_runtime': 3771.6531, 'train_samples_per_second': 74.455, 'train_steps_per_second': 4.656, 'total_flos': 6811884307405656.0, 'train_loss': 0.014386605999298852, 'epoch': 20.0})

In [23]:
model.save_pretrained("ner_model")

In [25]:
tokenizer.save_pretrained("ner_tokenizer")

('ner_tokenizer/tokenizer_config.json',
 'ner_tokenizer/special_tokens_map.json',
 'ner_tokenizer/vocab.txt',
 'ner_tokenizer/added_tokens.json',
 'ner_tokenizer/tokenizer.json')

Loading and inspecting the saved model and tokenizer

In [26]:
import json

In [27]:
config=json.load(open("ner_model/config.json"))
config

{'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4',
  '5': 'LABEL_5',
  '6': 'LABEL_6',
  '7': 'LABEL_7',
  '8': 'LABEL_8'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4,
  'LABEL_5': 5,
  'LABEL_6': 6,
  'LABEL_7': 7,
  'LABEL_8': 8},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.50.0',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [28]:
dataset["train"].features["ner_tags"].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [29]:
id_to_label = {str(i): label for i, label in enumerate(dataset["train"].features["ner_tags"].feature.names)}
label_to_id = {label: str(i) for i, label in enumerate(dataset["train"].features["ner_tags"].feature.names)}

print(id_to_label)
print(label_to_id)

{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC', '7': 'B-MISC', '8': 'I-MISC'}
{'O': '0', 'B-PER': '1', 'I-PER': '2', 'B-ORG': '3', 'I-ORG': '4', 'B-LOC': '5', 'I-LOC': '6', 'B-MISC': '7', 'I-MISC': '8'}


In [30]:
config["id2label"] = id_to_label
config["label2id"] = label_to_id

In [31]:
json.dump(config,open("/content/ner_model/config.json","w"))

Pushing the model and tokenizer to huggingface hub

In [32]:
!pip install huggingface_hub

In [34]:
from huggingface_hub import notebook_login
from transformers import AutoModelForTokenClassification, AutoTokenizer
from huggingface_hub import HfApi
notebook_login()

In [35]:
model_dir = "/content/ner_model"
tokenizer_dir = "/content/ner_tokenizer"
model_fine_tuned = AutoModelForTokenClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

In [36]:
# Push model to Hugging Face
model_fine_tuned.push_to_hub("Wothmag07/NER-fine-tuned-model")
tokenizer.push_to_hub("Wothmag07/NER-fine-tuned-model")

print("Model pushed to hugging-face successfully!")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model pushed to hugging-face successfully!


Testing on few data samples

In [38]:
ner_pipeline=pipeline("ner",model=model_fine_tuned,tokenizer=tokenizer)

Device set to use cuda:0


In [39]:
sample = "Elon Musk, the CEO of Tesla, announced that the company will open a new factory in Berlin, Germany. The announcement was made at a press conference in San Francisco on Tuesday."
ner_pipeline(sample)

[{'entity': 'B-PER',
  'score': np.float32(0.9999367),
  'index': 1,
  'word': 'el',
  'start': 0,
  'end': 2},
 {'entity': 'B-PER',
  'score': np.float32(0.9999498),
  'index': 2,
  'word': '##on',
  'start': 2,
  'end': 4},
 {'entity': 'I-PER',
  'score': np.float32(0.9999603),
  'index': 3,
  'word': 'mu',
  'start': 5,
  'end': 7},
 {'entity': 'I-PER',
  'score': np.float32(0.9999635),
  'index': 4,
  'word': '##sk',
  'start': 7,
  'end': 9},
 {'entity': 'B-ORG',
  'score': np.float32(0.9982967),
  'index': 9,
  'word': 'tesla',
  'start': 22,
  'end': 27},
 {'entity': 'B-LOC',
  'score': np.float32(0.9999589),
  'index': 21,
  'word': 'berlin',
  'start': 83,
  'end': 89},
 {'entity': 'B-LOC',
  'score': np.float32(0.99995935),
  'index': 23,
  'word': 'germany',
  'start': 91,
  'end': 98},
 {'entity': 'B-LOC',
  'score': np.float32(0.9999422),
  'index': 34,
  'word': 'san',
  'start': 151,
  'end': 154},
 {'entity': 'I-LOC',
  'score': np.float32(0.9998797),
  'index': 35,
  '

In [40]:
sample = "Apple Inc. reported a 10% increase in revenue for Q1 2025, with major contributions from iPhone 15 sales in the United States and China."
ner_pipeline(sample)

[{'entity': 'B-ORG',
  'score': np.float32(0.9999224),
  'index': 1,
  'word': 'apple',
  'start': 0,
  'end': 5},
 {'entity': 'I-ORG',
  'score': np.float32(0.9999038),
  'index': 2,
  'word': 'inc',
  'start': 6,
  'end': 9},
 {'entity': 'I-ORG',
  'score': np.float32(0.99990726),
  'index': 3,
  'word': '.',
  'start': 9,
  'end': 10},
 {'entity': 'B-MISC',
  'score': np.float32(0.99975115),
  'index': 21,
  'word': 'iphone',
  'start': 89,
  'end': 95},
 {'entity': 'I-MISC',
  'score': np.float32(0.9988483),
  'index': 22,
  'word': '15',
  'start': 96,
  'end': 98},
 {'entity': 'B-LOC',
  'score': np.float32(0.99993396),
  'index': 26,
  'word': 'united',
  'start': 112,
  'end': 118},
 {'entity': 'I-LOC',
  'score': np.float32(0.9998883),
  'index': 27,
  'word': 'states',
  'start': 119,
  'end': 125},
 {'entity': 'B-LOC',
  'score': np.float32(0.9999399),
  'index': 29,
  'word': 'china',
  'start': 130,
  'end': 135}]